In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import os

In [17]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kazanova/sentiment140")

print("Path to dataset files:", path)

Path to dataset files: /home/reynaldy/.cache/kagglehub/datasets/kazanova/sentiment140/versions/2


In [18]:
df = pd.read_csv(os.path.join("/home/reynaldy/.cache/kagglehub/datasets/kazanova/sentiment140/versions/2", "training.1600000.processed.noemoticon.csv"), encoding="latin1", header=None)
df

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [20]:
df = df.sample(frac=0.01, random_state=42)

In [21]:
df = df[[0,5]]
df

,0,5
541200,0,@chrishasboobs AHHH I HOPE YOUR OK!!!
750,0,"@misstoriblack cool , i have no tweet apps fo..."
766711,0,@TiannaChaos i know just family drama. its la...
285055,0,School email won't open and I have geography ...
705995,0,upper airways problem
...,...,...
1326992,4,On my way to my last exam.
1006015,4,"so, you should go check this guy out, he's pre..."
452359,0,enjoyed a wonderful weekend with my princess &...
449998,0,"got a sore throat well its not really sore, j..."


In [22]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import numpy as np
from sklearn.model_selection import train_test_split

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/reynaldy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/reynaldy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/reynaldy/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [23]:
df['label'] = df[0].apply(lambda x: 1 if x == 4 else 0)

In [24]:
print(df['label'].value_counts())

label
1    8018
0    7982
Name: count, dtype: int64


In [25]:
df["text"] = df[5]
df = df[["text", "label"]]
df.head(5)

,text,label
541200,@chrishasboobs AHHH I HOPE YOUR OK!!!,0
750,"@misstoriblack cool , i have no tweet apps fo...",0
766711,@TiannaChaos i know just family drama. its la...,0
285055,School email won't open and I have geography ...,0
705995,upper airways problem,0


In [26]:
# Split first into train, validation, and test

from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(
    df, 
    train_size=0.7, 
    random_state=42
)
    
# Second split: divide the remaining data into validation and test
relative_val_size = 0.15 / (0.15 + 0.15)
val_df, test_df = train_test_split(
    temp_df,
    train_size=relative_val_size,
    random_state=42
)

print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(val_df)}")
print(f"Test set size: {len(test_df)}")

Train set size: 11200
Validation set size: 2400
Test set size: 2400


Our goal is to fine tune a model from HuggingFace using LoRA or QLoRA

In [ ]:
from datasets import Dataset, load_dataset, DatasetDict
from huggingface_hub import HfApi, login
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments,
    BitsAndBytesConfig
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
import pandas as pd
import torch

login(token = "")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/reynaldy/.cache/huggingface/token
Login successful


In [28]:
# Try to upload the dataset on huggingface

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Create DatasetDict
from datasets import DatasetDict
dataset_dict = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

# Push to hub (auto update if already exist in hf_hub)
dataset_dict.push_to_hub("katsuchi/twitter_sentiment_analysis") # Create the repo first

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.67s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/katsuchi/twitter_sentiment_analysis/commit/f62835d175105978abaeb61eac3a2e63273f9cfb', commit_message='Upload dataset', commit_description='', oid='f62835d175105978abaeb61eac3a2e63273f9cfb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/katsuchi/twitter_sentiment_analysis', endpoint='https://huggingface.co', repo_type='dataset', repo_id='katsuchi/twitter_sentiment_analysis'), pr_revision=None, pr_num=None)

In [29]:
# Load back the dataset

dataset = load_dataset("katsuchi/twitter_sentiment_analysis")
dataset

Generating test split: 100%|██████████| 2400/2400 [00:00<00:00, 1473408.90 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 11200
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 2400
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 2400
    })
})

In [30]:
# Model and tokenization 

# Load base model and tokenizer
model_name = "distilbert/distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,  # Adjust based on your classification task
    torch_dtype=torch.float16
)

# Configure LoRA
lora_config = LoraConfig(
    r=1,  # rank
    lora_alpha=32,
    target_modules=["q_lin", "v_lin"],  # DistilBERT-specific attention modules
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

# Prepare model for LoRA
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 610,562 || all params: 67,565,572 || trainable%: 0.9037


In [46]:
from transformers import AutoModelForSequenceClassification, BitsAndBytesConfig

# Configure quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load new model and tokenizer 
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    quantization_config=bnb_config,
    torch_dtype=torch.float16
).to("cuda")

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query", "value"],  # BERT-specific attention modules
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

# Prepare model for LoRA
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

`low_cpu_mem_usage` was None, now set to True since model is quantized.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You shouldn't move a model that is dispatched using accelerate hooks.


trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700


In [32]:
print(model.config)

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method

In [33]:
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        return_tensors=None
    )

tokenized_dataset = {}
for split in dataset:
    tokenized_dataset[split] = dataset[split].map(
        tokenize_function,
        batched=True,
    )

tokenized_dataset

Map: 100%|██████████| 2400/2400 [00:00<00:00, 88717.49 examples/s]


{'train': Dataset({
     features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 11200
 }),
 'validation': Dataset({
     features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 2400
 }),
 'test': Dataset({
     features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 2400
 })}

In [34]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [42]:
# Training using Trainer Class from HuggingFace

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
        output_dir="bert-base-uncased-twitter-sentiment-analysis",
        num_train_epochs=2,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        learning_rate=2e-4,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=True,  # Set to False if you don't want to upload to HF Hub
        logging_steps=100,
        # Add warmup steps to prevent early overfitting
        warmup_ratio=0.1,
        # Add gradient clipping to prevent exploding gradients
        max_grad_norm=1.0,
        # Add early stopping
        metric_for_best_model="eval_loss",
        greater_is_better=False,
    )

/home/reynaldy/anaconda3/envs/unsloth_env/lib/python3.11/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [43]:
import evaluate

def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
    tokenizer = tokenizer,
    data_collator=data_collator
)

In [44]:
# Train the model
train_result = trainer.train()

# Evaluate the model
eval_result = trainer.evaluate()

# Print training metrics
print("\nTraining metrics:")
print(train_result.metrics)

# Print evaluation metrics
print("\nEvaluation metrics:")
print(eval_result)

/home/reynaldy/anaconda3/envs/unsloth_env/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.322800,0.557381,0.819583
2,0.351900,0.522728,0.819583


/home/reynaldy/anaconda3/envs/unsloth_env/lib/python3.11/site-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)



Training metrics:
{'train_runtime': 190.734, 'train_samples_per_second': 117.441, 'train_steps_per_second': 14.68, 'total_flos': 454113284247360.0, 'train_loss': 0.3117930841445923, 'epoch': 2.0}

Evaluation metrics:
{'eval_loss': 0.5227278470993042, 'eval_accuracy': 0.8195833333333333, 'eval_runtime': 8.7451, 'eval_samples_per_second': 274.439, 'eval_steps_per_second': 34.305, 'epoch': 2.0}


In [45]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/katsuchi/bert-base-uncased-twitter-sentiment-analysis/commit/50ee29080d083f4a32c037bb3b7b5d75f364bf46', commit_message='End of training', commit_description='', oid='50ee29080d083f4a32c037bb3b7b5d75f364bf46', pr_url=None, repo_url=RepoUrl('https://huggingface.co/katsuchi/bert-base-uncased-twitter-sentiment-analysis', endpoint='https://huggingface.co', repo_type='model', repo_id='katsuchi/bert-base-uncased-twitter-sentiment-analysis'), pr_revision=None, pr_num=None)

In [61]:
# Load the model

from peft import PeftModel
from transformers import AutoModelForSequenceClassification

base_model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")
model = PeftModel.from_pretrained(base_model, "katsuchi/bert-base-uncased-twitter-sentiment-analysis") # cuman Lora Adapter yang di load

inputs = "fk you"

input_ids = tokenizer(inputs, return_tensors="pt", padding=True, truncation=True, max_length=512)

print(input_ids)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[  101, 14352,  2017,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


In [62]:
with torch.no_grad():
    output = model(input_ids['input_ids'])


print(output.logits)

tensor([[-2.3177,  1.5544]])
